### SAMPLE PROCESSING FOR ENTITIES

This is a simple heuristic to leverage relations between entities in a sentence. We can refine it by using more knowledge about the relations. In this case, just measuring the path works. 

In [6]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import json 

url_DARPAs = "https://dsbox02.isi.edu:8888/wikidataos/bigdata/namespace/wdq/sparql"
url_PUBLIC = "https://query.wikidata.org/sparql"
url_PRIVATE = "http://wikidata-int.sl.cloud9.ibm.com:9999/bigdata/namespace/wdq/sparql"
sparql_PUB = SPARQLWrapper(url_PUBLIC)
sparql_PRIV = SPARQLWrapper(url_PRIVATE)

### EXAMPLE: "JAGUAR", assume "highway" is correct

query: “text”: “She noticed a Jaguar speeding on the highway”          
        “mention”: “Jaguar”         
        “predicted_entities”: [“Q35694”, “Q30055"]
        
The idea is to look a close path between the predicted_entities and "highway".  for node N such that Q30055 -R-> WD(N) -> WD("Highway") where WD(T) is the Wikidata translation of term T. Doesn't matter what R is, even though we may want to discriminate in the future. 

'Typical' metonimy relations in Conceptnet are: 

/r/AtLocation

/r/HasA

/r/HasProperty

/r/UsedFor

/r/PartOf

And the 'second tier' of relations: 

/r/Causes

/r/DefinedAs

/r/FormOf

/r/LocatedNear

In [55]:
#ALL OBJECTS IN A HIGHWAY
response = requests.get('http://api.conceptnet.io/query?end=/c/en/highway&rel=/r/AtLocation&limit=1000')
obj = response.json()
[edge['start']['@id'] for edge in obj['edges']]

['/c/en/car', '/c/en/lane', '/c/en/traffic_artery']

### Look up for the Wikidata references in Conceptnet
 'car' = (Q1420), 'lane' and 'traffic artery'. Repeat the query below for each of them. These are very fast queries. You can also try two path properties.

In [7]:
# The resource we want is http://wikidata.dbpedia.org/resource/Q1420
response = requests.get('http://api.conceptnet.io/query?start=/c/en/car&rel=/r/ExternalURL&limit=1000')
obj = response.json()
print([edge['end']['@id'] for edge in obj['edges']])


['http://wordnet-rdf.princeton.edu/wn31/102963937-n', 'http://sw.opencyc.org/2012/05/10/concept/en/Automobile', 'http://wikidata.dbpedia.org/resource/Q1420', 'http://dbpedia.org/resource/Car', 'http://wordnet-rdf.princeton.edu/wn31/102963378-n', 'http://wordnet-rdf.princeton.edu/wn31/102961779-n', 'http://wordnet-rdf.princeton.edu/wn31/102963788-n', 'http://umbel.org/umbel/sc/Automobile', 'http://en.wiktionary.org/wiki/CAR', 'http://en.wiktionary.org/wiki/car', 'http://fr.wiktionary.org/wiki/car', 'http://en.wiktionary.org/wiki/Car']


### Now check for paths between these entities and the predicted entities. 
We can use 2-level paths too, but in this case, only one hop is needed. 

In [52]:
#properties between 'car' and Q30055
query_text = """SELECT DISTINCT ?property1 ?property1Label  {
  wd:Q30055 ?property1 wd:Q1420.  
  SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}
 
  }"""
    
sparql_PUB.setQuery(query_text) 
sparql_PUB.setReturnFormat(JSON)
results = sparql_PUB.query().convert()

for result in results["results"]["bindings"]:       
       row = [result["property1"]["value"] , result["property1Label"]["value"]]
       print(row) 
       

['http://www.wikidata.org/prop/direct/P1056', 'http://www.wikidata.org/prop/direct/P1056']


In [3]:
#properties between 'car' and Q35694 -- There are no results
query_text = """SELECT DISTINCT ?property1 ?property1Label  {
  wd:Q35694 ?property1 wd:Q1420.  
  SERVICE wikibase:label {bd:serviceParam wikibase:language "en".}
 
  }"""
    
sparql_PUB.setQuery(query_text) 
sparql_PUB.setReturnFormat(JSON)
results = sparql_PUB.query().convert()

for result in results["results"]["bindings"]:       
       row = [result["property1"]["value"] , result["property1Label"]["value"]]
       print(row) 
       

### Check the parent of an entity

In [69]:
query_text = """ SELECT DISTINCT ?parent ?grandparent {
  # BIND (wd:Q30055 as ?entity)
  BIND (wd:Q30055 as ?entity). 
  
 {?entity wdt:P31|wdt:P279 ?parent . } UNION {
    ?entity (wdt:P31|wdt:P279) ?parent . 
    ?parent (wdt:P31|wdt:P279) ?grandparent
 }
 
  } ORDER BY ?parent  """

sparql_PUB.setQuery(query_text) 
sparql_PUB.setReturnFormat(JSON)
results = sparql_PUB.query().convert()

for result in results["results"]["bindings"]: 
       
       if "grandparent" in result : 
           row = [result["parent"]["value"] , result["grandparent"]["value"]]
       else:
           row = [result["parent"]["value"]]  
       print(row) 

['http://www.wikidata.org/entity/Q786820']
['http://www.wikidata.org/entity/Q786820', 'http://www.wikidata.org/entity/Q6881511']
['http://www.wikidata.org/entity/Q786820', 'http://www.wikidata.org/entity/Q13235160']


### Look up the entities mapped to 'jaguar'


In [10]:
import requests
obj = requests.get('http://api.conceptnet.io/c/en/highway').json()

In [11]:
obj['edges']

[{'@id': '/a/[/r/IsA/,/c/en/highway/,/c/en/road/]',
  '@type': 'Edge',
  'dataset': '/d/conceptnet/4/en',
  'end': {'@id': '/c/en/road',
   '@type': 'Node',
   'label': 'road',
   'language': 'en',
   'term': '/c/en/road'},
  'license': 'cc:by/4.0',
  'rel': {'@id': '/r/IsA', '@type': 'Relation', 'label': 'IsA'},
  'sources': [{'@id': '/and/[/s/activity/omcs/omcs1_possibly_free_text/,/s/contributor/omcs/delta407/]',
    '@type': 'Source',
    'activity': '/s/activity/omcs/omcs1_possibly_free_text',
    'contributor': '/s/contributor/omcs/delta407'},
   {'@id': '/and/[/s/activity/omcs/omcs1_possibly_free_text/,/s/contributor/omcs/jerryobject/]',
    '@type': 'Source',
    'activity': '/s/activity/omcs/omcs1_possibly_free_text',
    'contributor': '/s/contributor/omcs/jerryobject'},
   {'@id': '/and/[/s/activity/omcs/omcs1_possibly_free_text/,/s/contributor/omcs/whitten/]',
    '@type': 'Source',
    'activity': '/s/activity/omcs/omcs1_possibly_free_text',
    'contributor': '/s/contribu

In [16]:
#properties associated to Highway
response = requests.get('http://api.conceptnet.io/query?start=/c/en/highway&limit=1000')
obj = response.json()
[edge['rel']['@id'] + edge['end']['@id'] for edge in obj['edges']]

['/r/IsA/c/en/road',
 '/r/UsedFor/c/en/travel',
 '/r/UsedFor/c/en/drive_on',
 '/r/AtLocation/c/en/between_to_cities',
 '/r/UsedFor/c/en/go_elsewhere',
 '/r/AtLocation/c/en/country',
 '/r/RelatedTo/c/en/road',
 '/r/UsedFor/c/en/motor_vehicles_to_travel_on',
 '/r/IsA/c/en/road/n/wn/artifact',
 '/r/IsA/c/en/australian_freeway',
 '/r/AtLocation/c/en/city',
 '/r/AtLocation/c/en/atlas',
 '/r/Synonym/c/sh/autoput',
 '/r/UsedFor/c/en/driving_on',
 '/r/UsedFor/c/en/travel_in_car',
 '/r/UsedFor/c/en/getting_from_one_place_to_another',
 '/r/Synonym/c/vi/quốc_lộ',
 '/r/Synonym/c/gag/şösä',
 '/r/Synonym/c/ka/გზატკეცილი',
 '/r/Synonym/c/he/כביש_מהיר',
 '/r/Synonym/c/ar/طريق_عام',
 '/r/UsedFor/c/en/traveling_long_distances_quickly',
 '/r/UsedFor/c/en/crossing_country',
 '/r/Synonym/c/sv/väg',
 '/r/UsedFor/c/en/improving_transportation',
 '/r/AtLocation/c/en/developed_land',
 '/r/Synonym/c/da/motorvej',
 '/r/Synonym/c/ms/lebuh_raya',
 '/r/Synonym/c/hy/խճուղի',
 '/r/UsedFor/c/en/riding_bus',
 '/r/Synon